In [1]:
# import dependencies
import pandas as pd
import requests
import numpy as np
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import sqlite3

In [2]:
# create database connection
db = '../database\sqlite_db.sqlite'

conn = sqlite3.connect(db)
conn.row_factory = sqlite3.Row

c = conn.cursor()

In [3]:
# grab the team abbreviations for url building and placing them in a datframe
table = 'Teams'
query = f'SELECT * FROM {table}'

df = pd.DataFrame(c.execute(query))
df.columns = list(map(lambda x: x[0], c.description))
df.head()

,ID,Name,Abbreviation,Conference,Division
0,1.0,Arizona Cardinals,ARI,NFC,West
1,2.0,Atlanta Falcons,ATL,NFC,South
2,3.0,Baltimore Ravens,BAL,AFC,North
3,4.0,Buffalo Bills,BUF,AFC,East
4,5.0,Carolina Panthers,CAR,NFC,South


In [4]:
# creating a list of all team abbreviations
teams = df['Abbreviation'].tolist()
teams[0] = 'ARZ'

In [5]:
# creating dictionary of archive dates and their archive number
archive_num = {'2018': '212',
              '2019': '224',
              '2020': '236',
              '2021': '249'}
years = list(archive_num.keys())
archive = list(archive_num.values())

In [6]:
# Intitialze web driver
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(options=options)

In [7]:
base_url = 'https://www.ourlads.com/nfldepthcharts/archive/'

In [8]:
df_depth_chart = pd.DataFrame(columns=['team', 'year', 'position', 'depth_1', 'depth_2', 'depth_3', 'depth_4', 'depth_5'])

In [9]:
def determine_depth(row):
    
    position = row[0].text
    depth_1 = row[2].text
    depth_2 = row[4].text
    depth_3 = row[6].text
    depth_4 = row[8].text
    depth_5 = row[10].text
    
    depth ={'position': position,
            'depth_1': depth_1, 
            'depth_2': depth_2, 
            'depth_3': depth_3, 
            'depth_4': depth_4, 
            'depth_5': depth_5}
    
    return depth

In [10]:
def clear_dict():
    del table_row['position']
    del table_row['depth_1']
    del table_row['depth_2']
    del table_row['depth_3']
    del table_row['depth_4']
    del table_row['depth_5']

In [11]:
count = 0
# nested loop to go to archive year then loop through all teams and grab the data, before moving to the next year\
for year in archive:
    table_row = {'year': years[count]}
        
    print(f'year is {table_row}')
    for team in teams:
        # add team to dictionary
        table_row['team'] = team
        print(f'team: {team}')
        
        # go to constructed archived depth chart
        driver.get(f'{base_url}{year}/{team}')
        print('found page, scraping')
        sleep(2)
        
        # create soup environment
        html = driver.page_source
        page = soup(html, 'html.parser')
        
        #grabbing both halves of the data since they are split along row color
        page_table = page.find('tbody', id='ctl00_phContent_dcTBody')
        white_results = page_table.find_all('tr', class_='row-dc-wht')
        gray_results = page_table.find_all('tr', class_='row-dc-grey')
        
        # lopping through results
        for result in white_results:
            depth_data = determine_depth(result.find_all('td'))
            table_row.update(depth_data)
            # appending through the dataframe
            df_depth_chart = df_depth_chart.append(table_row, ignore_index=True)
            print(f'data: {table_row}')
            clear_dict()
        for result in gray_results:
            depth_data = determine_depth(result.find_all('td'))
            table_row.update(depth_data)
            df_depth_chart = df_depth_chart.append(table_row, ignore_index=True)
            print(f'data: {table_row}')
            clear_dict()
    count +=1

year is {'year': '2018'}
team: ARZ
found page, scraping
data: {'year': '2018', 'team': 'ARZ', 'position': 'LWR', 'depth_1': 'FITZGERALD, LARRY 04/1', 'depth_2': 'Williams, Chad 17/3', 'depth_3': 'Kirk, Christian 18/2', 'depth_4': 'Sherfield, Trent CF18', 'depth_5': 'Duncan, CJ CF18'}
data: {'year': '2018', 'team': 'ARZ', 'position': 'RWR', 'depth_1': '', 'depth_2': '', 'depth_3': '', 'depth_4': '', 'depth_5': 'Willis, Corey CF18'}
data: {'year': '2018', 'team': 'ARZ', 'position': 'LG', 'depth_1': 'IUPATI, MIKE U/SF', 'depth_2': 'Boehm, Evan 16/4', 'depth_3': 'House, Will CF18', 'depth_4': '', 'depth_5': ''}
data: {'year': '2018', 'team': 'ARZ', 'position': 'RG', 'depth_1': 'Pugh, Justin U/NYG', 'depth_2': 'Allen, Josh SF18', 'depth_3': 'Pyke, Greg SF18', 'depth_4': '', 'depth_5': ''}
data: {'year': '2018', 'team': 'ARZ', 'position': 'TE', 'depth_1': 'Seals-Jones, Ricky CF17', 'depth_2': 'Holmes, Gabe SF17', 'depth_3': 'Bloom, Alec CF18', 'depth_4': 'Vollert, Andrew CF18', 'depth_5': 'W

In [12]:
# grabbing latest 2022 data
base_url = 'https://www.ourlads.com/nfldepthcharts/depthchart/'
table_row = {'year': '2022'}
for team in teams:
    # add team to dictionary
    table_row['team'] = team
    print(f'team: {team}')

    # go to constructed archived depth chart
    driver.get(f'{base_url}{team}')
    print('found page, scraping')
    sleep(2)

    # create soup environment
    html = driver.page_source
    page = soup(html, 'html.parser')

    #grabbing both halves of the data since they are split along row color
    page_table = page.find('tbody', id='ctl00_phContent_dcTBody')
    white_results = page_table.find_all('tr', class_='row-dc-wht')
    gray_results = page_table.find_all('tr', class_='row-dc-grey')

    # lopping through results
    for result in white_results:
        depth_data = determine_depth(result.find_all('td'))
        table_row.update(depth_data)
        # appending through the dataframe
        df_depth_chart = df_depth_chart.append(table_row, ignore_index=True)
        print(f'data: {table_row}')
        clear_dict()
    for result in gray_results:
        depth_data = determine_depth(result.find_all('td'))
        table_row.update(depth_data)
        df_depth_chart = df_depth_chart.append(table_row, ignore_index=True)
        print(f'data: {table_row}')
        clear_dict()

team: ARZ
found page, scraping
data: {'year': '2022', 'team': 'ARZ', 'position': 'LWR', 'depth_1': 'GREEN, A.J. U/Cin', 'depth_2': 'Wesley, Antoine U/Bal', 'depth_3': 'Blake, Christian U/Atl', 'depth_4': 'Kirklin, Jontre CF22', 'depth_5': ''}
data: {'year': '2022', 'team': 'ARZ', 'position': 'SWR', 'depth_1': 'Moore, Rondale 21/2', 'depth_2': 'Dortch, Greg SF21', 'depth_3': 'Payton, JaVonta CF22', 'depth_4': '', 'depth_5': ''}
data: {'year': '2022', 'team': 'ARZ', 'position': 'LG', 'depth_1': 'PUGH, JUSTIN U/NYG', 'depth_2': 'Hayes, Marquis 22/7', 'depth_3': 'Harlow, Sean SF21', 'depth_4': '', 'depth_5': ''}
data: {'year': '2022', 'team': 'ARZ', 'position': 'RG', 'depth_1': 'Hernandez, Will U/NYG', 'depth_2': 'Murray, Justin W/LV', 'depth_3': 'Isidora, Danny SF21', 'depth_4': 'Long, Greg CF22', 'depth_5': ''}
data: {'year': '2022', 'team': 'ARZ', 'position': 'TE', 'depth_1': 'ERTZ, ZACH T/Phi', 'depth_2': 'McBride, Trey 22/2', 'depth_3': 'Williams, Maxx PUP', 'depth_4': 'Anderson, Step

In [13]:
driver.quit()

In [14]:
df_depth_chart

,team,year,position,depth_1,depth_2,depth_3,depth_4,depth_5
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1","Williams, Chad 17/3","Kirk, Christian 18/2","Sherfield, Trent CF18","Duncan, CJ CF18"
1,ARZ,2018,RWR,,,,,"Willis, Corey CF18"
2,ARZ,2018,LG,"IUPATI, MIKE U/SF","Boehm, Evan 16/4","House, Will CF18",,
3,ARZ,2018,RG,"Pugh, Justin U/NYG","Allen, Josh SF18","Pyke, Greg SF18",,
4,ARZ,2018,TE,"Seals-Jones, Ricky CF17","Holmes, Gabe SF17","Bloom, Alec CF18","Vollert, Andrew CF18","Williams, Bryce SF18"
...,...,...,...,...,...,...,...,...
5239,WAS,2022,SS,"Curl, Kamren 20/7","Forrest, Darrick 21/5","Apke, Troy 18/4",,
5240,WAS,2022,RCB,"Jackson III, William U/Cin","Neal, DeJuan SF22","Drayden, Josh CF22",,
5241,WAS,2022,PK,"Slye, Joey SF21",,,,
5242,WAS,2022,H,"WAY, TRESS W/Chi",,,,


In [15]:
df_depth_chart.position.unique()

array(['LWR', 'RWR', 'LG', 'RG', 'TE', 'QB', 'FB', 'LDE', 'DT', 'SLB',
       'WLB', 'SS', 'RCB', 'P', 'LS', 'PR', 'KO', 'RES', 'LT', 'C', 'RT',
       'RB', 'RDE', 'MLB', 'LCB', 'FS', 'PK', 'H', 'KR', 'PS', 'SWR',
       'DE', 'NT', 'RUSH', 'LE', 'RE', 'RDT', 'LDT', 'LILB', 'ROLB', 'S',
       'LOLB', 'RILB', 'TE/HB', 'NB', 'ILB', 'NLB', 'OLB', 'RET', 'WILL',
       'JACK', 'MIKE', 'LB', 'SILB', 'SAM', 'DE/OB', 'PT', 'SUS', 'NCB',
       'DB', 'DNR', 'CB', 'UT', 'SCB', 'LEO', 'DL', 'LOB', 'ROB', 'WR',
       'EDGE'], dtype=object)

In [16]:
positions = ['QB', 'RB', 'TE', 'LWR', 'RWR','FB', 'TE/HB']
df_depth_chart = df_depth_chart[df_depth_chart['position'].isin(positions)]

df_depth_chart

,team,year,position,depth_1,depth_2,depth_3,depth_4,depth_5
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1","Williams, Chad 17/3","Kirk, Christian 18/2","Sherfield, Trent CF18","Duncan, CJ CF18"
1,ARZ,2018,RWR,,,,,"Willis, Corey CF18"
4,ARZ,2018,TE,"Seals-Jones, Ricky CF17","Holmes, Gabe SF17","Bloom, Alec CF18","Vollert, Andrew CF18","Williams, Bryce SF18"
5,ARZ,2018,QB,"BRADFORD, SAM U/Min","Rosen, Josh 18/1","Glennon, Mike CC/Chi","Kanoff, Chad CF18",
6,ARZ,2018,FB,"Coleman, Derrick SF18","Penny, Elijhaa CF16",,,
...,...,...,...,...,...,...,...,...
5218,WAS,2022,RB,,,,,"Bonnafon, Reggie SF22"
5229,WAS,2022,RWR,"Dotson, Jahan 22/1","Brown, Dyami 21/3","Michel, Marken SF21",,
5233,WAS,2022,TE,,,,"Rogers, Armani CF22","Hodges, Curtis CF22"
5234,WAS,2022,RB,"Gibson, Antonio 20/3","McKissic, J.D. U/Det","Robinson, Brian 22/3","Patterson, Jaret CF21","Williams, Jonathan P/NYG"


In [17]:
df_depth_1 = df_depth_chart.drop(columns=['depth_2', 'depth_3', 'depth_4', 'depth_5'])
df_depth_2 = df_depth_chart.drop(columns=['depth_1', 'depth_3', 'depth_4', 'depth_5'])
df_depth_3 = df_depth_chart.drop(columns=['depth_1', 'depth_2', 'depth_4', 'depth_5'])
df_depth_4 = df_depth_chart.drop(columns=['depth_1', 'depth_2', 'depth_3', 'depth_5'])
df_depth_5 = df_depth_chart.drop(columns=['depth_1', 'depth_2', 'depth_3', 'depth_4'])

In [18]:
df_depth_1.columns

Index(['team', 'year', 'position', 'depth_1'], dtype='object')

In [19]:
column_names = ['team', 'year', 'position', 'name']

df_depth_1.columns = column_names
df_depth_2.columns = column_names
df_depth_3.columns = column_names
df_depth_4.columns = column_names
df_depth_5.columns = column_names

In [20]:
df_depth_1.columns

Index(['team', 'year', 'position', 'name'], dtype='object')

In [21]:
df_depth_1

,team,year,position,name
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1"
1,ARZ,2018,RWR,
4,ARZ,2018,TE,"Seals-Jones, Ricky CF17"
5,ARZ,2018,QB,"BRADFORD, SAM U/Min"
6,ARZ,2018,FB,"Coleman, Derrick SF18"
...,...,...,...,...
5218,WAS,2022,RB,
5229,WAS,2022,RWR,"Dotson, Jahan 22/1"
5233,WAS,2022,TE,
5234,WAS,2022,RB,"Gibson, Antonio 20/3"


In [22]:
df_depth_1['depth'] = 1
df_depth_2['depth'] = 2
df_depth_3['depth'] = 3
df_depth_4['depth'] = 4
df_depth_5['depth'] = 5

In [23]:
df_depth_1

,team,year,position,name,depth
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1",1
1,ARZ,2018,RWR,,1
4,ARZ,2018,TE,"Seals-Jones, Ricky CF17",1
5,ARZ,2018,QB,"BRADFORD, SAM U/Min",1
6,ARZ,2018,FB,"Coleman, Derrick SF18",1
...,...,...,...,...,...
5218,WAS,2022,RB,,1
5229,WAS,2022,RWR,"Dotson, Jahan 22/1",1
5233,WAS,2022,TE,,1
5234,WAS,2022,RB,"Gibson, Antonio 20/3",1


In [24]:
df_depth_chart = df_depth_1.append(df_depth_2, ignore_index=True)
df_depth_chart

,team,year,position,name,depth
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1",1
1,ARZ,2018,RWR,,1
2,ARZ,2018,TE,"Seals-Jones, Ricky CF17",1
3,ARZ,2018,QB,"BRADFORD, SAM U/Min",1
4,ARZ,2018,FB,"Coleman, Derrick SF18",1
...,...,...,...,...,...
2193,WAS,2022,RB,,2
2194,WAS,2022,RWR,"Brown, Dyami 21/3",2
2195,WAS,2022,TE,,2
2196,WAS,2022,RB,"McKissic, J.D. U/Det",2


In [25]:
df_depth_chart = df_depth_chart.append(df_depth_3, ignore_index=True)
df_depth_chart = df_depth_chart.append(df_depth_4, ignore_index=True)
df_depth_chart = df_depth_chart.append(df_depth_5, ignore_index=True)
df_depth_chart

,team,year,position,name,depth
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1",1
1,ARZ,2018,RWR,,1
2,ARZ,2018,TE,"Seals-Jones, Ricky CF17",1
3,ARZ,2018,QB,"BRADFORD, SAM U/Min",1
4,ARZ,2018,FB,"Coleman, Derrick SF18",1
...,...,...,...,...,...
5490,WAS,2022,RB,"Bonnafon, Reggie SF22",5
5491,WAS,2022,RWR,,5
5492,WAS,2022,TE,"Hodges, Curtis CF22",5
5493,WAS,2022,RB,"Williams, Jonathan P/NYG",5


In [26]:
df_depth_chart.replace('', np.nan, inplace=True)
df_depth_chart.dropna(how='any', inplace=True)
df_depth_chart

,team,year,position,name,depth
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1",1
2,ARZ,2018,TE,"Seals-Jones, Ricky CF17",1
3,ARZ,2018,QB,"BRADFORD, SAM U/Min",1
4,ARZ,2018,FB,"Coleman, Derrick SF18",1
5,ARZ,2018,RWR,"Butler, Brice U/Dal",1
...,...,...,...,...,...
5485,TEN,2022,RB,"Wilkins, Jordan SF21",5
5488,WAS,2022,TE,"Gandy-Golden, Antonio 20/4",5
5490,WAS,2022,RB,"Bonnafon, Reggie SF22",5
5492,WAS,2022,TE,"Hodges, Curtis CF22",5


In [27]:
import re
for index, row in df_depth_chart.iterrows():
    name_list = row['name'].rsplit(maxsplit=1)
    name = name_list[0]
    df_depth_chart.loc[index, 'name_cleaned'] = name
    
df_depth_chart

,team,year,position,name,depth,name_cleaned
0,ARZ,2018,LWR,"FITZGERALD, LARRY 04/1",1,"FITZGERALD, LARRY"
2,ARZ,2018,TE,"Seals-Jones, Ricky CF17",1,"Seals-Jones, Ricky"
3,ARZ,2018,QB,"BRADFORD, SAM U/Min",1,"BRADFORD, SAM"
4,ARZ,2018,FB,"Coleman, Derrick SF18",1,"Coleman, Derrick"
5,ARZ,2018,RWR,"Butler, Brice U/Dal",1,"Butler, Brice"
...,...,...,...,...,...,...
5485,TEN,2022,RB,"Wilkins, Jordan SF21",5,"Wilkins, Jordan"
5488,WAS,2022,TE,"Gandy-Golden, Antonio 20/4",5,"Gandy-Golden, Antonio"
5490,WAS,2022,RB,"Bonnafon, Reggie SF22",5,"Bonnafon, Reggie"
5492,WAS,2022,TE,"Hodges, Curtis CF22",5,"Hodges, Curtis"


In [28]:
df_depth_chart.drop(columns='name', inplace=True)
df_depth_chart

,team,year,position,depth,name_cleaned
0,ARZ,2018,LWR,1,"FITZGERALD, LARRY"
2,ARZ,2018,TE,1,"Seals-Jones, Ricky"
3,ARZ,2018,QB,1,"BRADFORD, SAM"
4,ARZ,2018,FB,1,"Coleman, Derrick"
5,ARZ,2018,RWR,1,"Butler, Brice"
...,...,...,...,...,...
5485,TEN,2022,RB,5,"Wilkins, Jordan"
5488,WAS,2022,TE,5,"Gandy-Golden, Antonio"
5490,WAS,2022,RB,5,"Bonnafon, Reggie"
5492,WAS,2022,TE,5,"Hodges, Curtis"


In [29]:
df_depth_chart.sample(25)

,team,year,position,depth,name_cleaned
962,IND,2022,LWR,1,"Pittman Jr., Michael"
2700,CLE,2020,RWR,3,"Taylor, Taywan"
2303,HOU,2018,TE,3,"Thomas, Jordan"
2538,IND,2019,RWR,3,"Cain, Deon"
3601,CLE,2019,TE,4,"Seals-Jones, Ricky"
2811,PHI,2020,LWR,3,"Burnett, Deontay"
4955,KC,2020,RWR,5,"Shelton-Mosley, Justice"
723,CLE,2021,RB,1,"Chubb, Nick"
3068,ARZ,2022,RWR,3,"Baccellia, Andre"
798,NYG,2021,TE,1,"Smith, Kaden"


In [30]:
for index, row in df_depth_chart.iterrows():
    name_list = row['name_cleaned'].split(', ')
    last_name = name_list[0]
    first_name = name_list[1]
    
    df_depth_chart.loc[index, 'first_name'] = first_name
    df_depth_chart.loc[index, 'last_name'] = last_name


In [31]:
df_depth_chart

,team,year,position,depth,name_cleaned,first_name,last_name
0,ARZ,2018,LWR,1,"FITZGERALD, LARRY",LARRY,FITZGERALD
2,ARZ,2018,TE,1,"Seals-Jones, Ricky",Ricky,Seals-Jones
3,ARZ,2018,QB,1,"BRADFORD, SAM",SAM,BRADFORD
4,ARZ,2018,FB,1,"Coleman, Derrick",Derrick,Coleman
5,ARZ,2018,RWR,1,"Butler, Brice",Brice,Butler
...,...,...,...,...,...,...,...
5485,TEN,2022,RB,5,"Wilkins, Jordan",Jordan,Wilkins
5488,WAS,2022,TE,5,"Gandy-Golden, Antonio",Antonio,Gandy-Golden
5490,WAS,2022,RB,5,"Bonnafon, Reggie",Reggie,Bonnafon
5492,WAS,2022,TE,5,"Hodges, Curtis",Curtis,Hodges


In [32]:
for index, row in df_depth_chart.iterrows():
    name = f"{row['first_name']} {row['last_name']}"
    df_depth_chart.loc[index, 'name'] = name
    
df_depth_chart.sample(50)

,team,year,position,depth,name_cleaned,first_name,last_name,name
4325,LV,2022,TE,4,"Bowers, Nick",Nick,Bowers,Nick Bowers
346,JAX,2019,TE,1,"Swaim, Geoff",Geoff,Swaim,Geoff Swaim
3930,SF,2020,LWR,4,"Poindexter, Shawn",Shawn,Poindexter,Shawn Poindexter
4281,MIA,2022,LWR,4,"Sanders, Braylon",Braylon,Sanders,Braylon Sanders
1788,BAL,2021,TE,2,"Oliver, Josh",Josh,Oliver,Josh Oliver
1021,NYJ,2022,QB,1,"Wilson, Zach",Zach,Wilson,Zach Wilson
1470,NE,2019,RB,2,"Burkhead, Rex",Rex,Burkhead,Rex Burkhead
2024,DAL,2022,LWR,2,"Fehoko, Simi",Simi,Fehoko,Simi Fehoko
514,DEN,2020,TE,1,"Vannett, Nick",Nick,Vannett,Nick Vannett
1758,TB,2020,RB,2,"Jones II, Ronald",Ronald,Jones II,Ronald Jones II


In [33]:
df_depth_chart.drop(columns=['name_cleaned', 'first_name', 'last_name'], inplace=True)

In [34]:
df_depth_chart

,team,year,position,depth,name
0,ARZ,2018,LWR,1,LARRY FITZGERALD
2,ARZ,2018,TE,1,Ricky Seals-Jones
3,ARZ,2018,QB,1,SAM BRADFORD
4,ARZ,2018,FB,1,Derrick Coleman
5,ARZ,2018,RWR,1,Brice Butler
...,...,...,...,...,...
5485,TEN,2022,RB,5,Jordan Wilkins
5488,WAS,2022,TE,5,Antonio Gandy-Golden
5490,WAS,2022,RB,5,Reggie Bonnafon
5492,WAS,2022,TE,5,Curtis Hodges


In [35]:
df_depth_chart.to_sql('DepthCharts', conn, if_exists='replace')

In [36]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""